In [ ]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
                   sep=";")

In [4]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [7]:
#data split into train, val, test

train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(['quality'], axis = 1).values
train_y = train[['quality']].values.ravel()
test_x = test.drop(['quality'], axis = 1).values
test_y = test[['quality']].values.ravel()

train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

signature = infer_signature(train_x, train_y)

## ANN Model

In [21]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [22]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [23]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [24]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1:00 1s/step - loss: 37.5346 - root_mean_squared_error: 6.1266
12/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.5936 - root_mean_squared_error: 5.3293 
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 22.1132 - root_mean_squared_error: 4.6401
39/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 17.7430 - root_mean_squared_error: 4.1042
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 16.1962 - root_mean_squared_error: 3.8985
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 16.0031 - root_mean_squared_error: 3.8723 - val_loss: 0.5734 - val_root_mean_squared_error: 0.7572

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.5977 - root_mean_squared_error: 0.7731
14/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6117 - root_mean_squared_error: 0.7820 
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6046 - root_mean_squared_error: 0.7775
40/46 ━━━━━━━━━━━━━━━━━━━━ 0

2025/07/04 01:18:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 9s 200ms/step - loss: 34.1423 - root_mean_squared_error: 5.8431
13/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 34.3536 - root_mean_squared_error: 5.8611  
28/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 34.5953 - root_mean_squared_error: 5.8817
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 34.7465 - root_mean_squared_error: 5.8945
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 34.7925 - root_mean_squared_error: 5.8984 - val_loss: 35.6671 - val_root_mean_squared_error: 5.9722

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 34.8056 - root_mean_squared_error: 5.8996
16/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 34.6857 - root_mean_squared_error: 5.8894 
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.8307 - root_mean_squared_error: 5.9017
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.9387 - root

2025/07/04 01:19:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 10s 225ms/step - loss: 36.2615 - root_mean_squared_error: 6.0218
12/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 35.5855 - root_mean_squared_error: 5.9645   
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 33.4727 - root_mean_squared_error: 5.7820
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31.4515 - root_mean_squared_error: 5.5997
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.2553 - root_mean_squared_error: 5.4879
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 30.1138 - root_mean_squared_error: 5.4745 - val_loss: 11.1521 - val_root_mean_squared_error: 3.3395

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 11.2269 - root_mean_squared_error: 3.3507
16/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.4261 - root_mean_squared_error: 3.0671  
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.2874 - roo

...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 10s 224ms/step - loss: 39.1375 - root_mean_squared_error: 6.2560
15/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 36.1151 - root_mean_squared_error: 6.0090   
30/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.6179 - root_mean_squared_error: 5.9676
44/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.3417 - root_mean_squared_error: 5.9445
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 35.2923 - root_mean_squared_error: 5.9403 - val_loss: 33.9864 - val_root_mean_squared_error: 5.8298

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 32.6356 - root_mean_squared_error: 5.7128
12/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 33.6340 - root_mean_squared_error: 5.7994 
24/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 33.6249 - root_mean_squared_error: 5.7986
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 33.5450 - ro

2025/07/04 01:19:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:27<00:00,  6.97s/trial, best loss: 0.7550686001777649]

2025/07/04 01:19:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.010425865500899478), 'momentum': np.float64(0.1648964198774655)}
Best eval rmse: 0.7550686001777649
